---

---

# Predição sonar (rocha ou mina)

Este notebook implementa uma rede neural para predizer se um objeto é uma mina ou uma rocha com base em sinais de um sonar opbtidos em diferentes ângulos.

Trata-se de um problema de classificação binária (R-rocha ou M-mina).

---



---




## Conjunto de dados

- Fonte: https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)

O conjunto de dados contém sinais obtidos de uma variedade de ângulos  diferentes. Cada padrão é um conjunto de 60 números no intervalo de 0,0 a 1,0. Cada número representa a energia dentro de uma determinada banda de frequência, integrada ao longo de um determinado período de tempo.

Detalhes sobre o conjunto de dados:

1. Número de instâncias: 208

2. Número de atributos: 60 

4. Variável target (classe): o rótulo associado a cada registro contém a letra “R” se o objeto for uma rocha e “M” se for uma mina (cilindro de metal)

 

## Leitura e preparação dos dados

Vamos começar importando o arquivo CSV bruto usando o Pandas.

In [ ]:
import pandas as pd

# Clone do repositório de dados do GitHub
!git clone https://github.com/malegopc/AM2PUCPOC
# lê arquivo de dados, atribue NaN para dados faltantes e informa cabeçalho como inexistentes (sem nomes para os atributos)
sonar = pd.read_csv('/content/AM2PUCPOC/Datasets/Sonar/sonar.all-data.csv', na_values=['?'], header = None)
# imprime as 5 primeiras linha dos dados montados
sonar.head()

## Análise prévia dos dados

Imprime o número de dados non-null (não ausentes) e o tipo de cada atributo.

In [ ]:
# Mostra o número de dados non-null (não ausentes) e o tipo de cada atributo
print('Informações sobre os dados:\n')
sonar.info()
# Soma o número de dados null (ausentes)
print('\n\nSoma de dados null (vazios):\n')
sonar.isnull().sum()

## Transforma dados categóricos em números 

- M => 0 (mina)
- R => 1 (rocha)

In [ ]:
sonar.replace(('R', 'M'), (1, 0), inplace=True)
sonar.head()

## Análise (estatística) descritiva dos dados

Análise descritiva dos dados (resumo).

In [ ]:
sonar.describe()

## Separa os atributos das classes

Extrai os atributos e as classes (rótulos) colocando-os em duas variáveis separadas (na forma que o Keras espera).

Observe que o Dataframe "sonar" possui 61 colunas.

In [ ]:
sonar = sonar.values
print(type(sonar))
X = sonar[:,0:60].astype(float) # X recebe todas as colunas exceto a última
y = sonar[:,60] # y recebe a última coluna
print(X.shape)
print(y.shape)
print("\nX = \n",X)
print("\ny = \n", y)

## Divide o conjunto de dados em treino e teste

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

## Normaliza os dados

StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Criar o modelo de rede neural



## Compilar o modelo




## Treinar o modelo



## Avaliação do modelo

Alternativamente você pode utilizar de maneira isolada a função **evaluate** para avaliar seu modelo no conjunto de dados desejado.

A função **evaluate()** retornará uma lista com dois valores. O primeiro será a perda do modelo no conjunto de dados e o segundo será a precisão do modelo no conjunto de dados.

Seus resultados podem variar devido à natureza estocástica do algoritmo. As redes neurais são um algoritmo estocástico, o que significa que o mesmo algoritmo nos mesmos dados pode gerar resultados diferentes.

In [ ]:
model.evaluate(X_test, y_test)

## Análise da função loss (erro/perda)
Podemos observar o comportamento da função loss para os dados de treino e de validação plotando um gráfico do histórico do comportamento do valor desta função durante o treinamento.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'], '')
plt.xlabel("Épocas")
plt.ylabel('Loss')
plt.title('Função de erro/loss')
plt.legend(['loss', 'val_loss'])
plt.show()

## Análise da métrica de desempenho (acurácia)

Podemos também observar o comportamento da precisão do modelo para os dados de treino e de validação plotando um gráfico do histórico do comportamento desta medida durante o treinamento.

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'], '')
plt.xlabel("Épocas")
plt.ylabel('Acurácia')
plt.title('Acurácia durante o treinamento')
plt.legend(['accuracy', 'val_accuracy'])
plt.show()

## Fazendo predições

É possível fazer predições (classificações) utilizando o modelo neural treinado sobre as amostras do conjunto de dados de treinamento fingindo que é um novo conjunto de dados que não vimos antes, sobre o conjunto teste ou qualquer outro conjunto novo.

Para isso, basta chamar a função **predict()**. 

Como estamos usando ua função de ativação sigmóide na camada de saída, então as previsões serão uma probabilidade na faixa entre 0 e 1. Podemos facilmente convertê-las em uma previsão binária nítida para esta tarefa de classificação arredondando-as.



In [ ]:
import numpy as np
from random import randint
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
# imprime as predições para as 10 primeiras amostras do conjunto de dados
for i in range(10):
	print('instância no. %d => %d (expected %d)' % (i, y_pred[i], y_test[i]))

## Calcula a matriz de confusão

Para se ter uma noção melhor dos resultados das predições realizadas pelo modelo neural treinado pode-se calcular a [**matriz de confusão**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) sobre os resultados das classificações.

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)